<a href="https://colab.research.google.com/github/l-Monarch-l/Laborat/blob/main/%D0%9C%D0%9E%D0%A1%D0%98_%D0%BB%D0%B0%D0%B1_2.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Установка библиотек

In [2]:
!pip install nltk pymorphy3 natasha scikit-learn

  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.8/53.8 kB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 34.4/34.4 MB 37.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 46.7/46.7 kB 4.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 55.5/55.5 kB 5.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.4/8.4 MB 99.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 8.2/8.2 MB 118.4 MB/s eta 0:00:00
  Created wheel for docopt: filename=docopt-0.6.2-py2.py3-none-any.whl size=13706 sha256=e38c35b688c53304a28425344617e12102dad24d8c1f5f05a5edb25b1f8803ef
  Stored in directory: /root/.cache/pip/wheels/1a/b0/8c/4b75c4116c31f83c8f9f047231251e13cc74481cca4a78a9ce
  Created wheel for intervaltree: filename=intervaltree-3.1.0-py2.py3-none-any.whl size=26097 sha256=a9a87941193a4d38f3fcbab63c8918cc07124a7710b03915acf5d51689dab288
  Stored in directory: 

Импортируем необходимые библиотеки

In [11]:
import re
from pymorphy3 import MorphAnalyzer
import math

Подготовка данных, то бишь подготовили текст и список стоп-слов

In [6]:
# Инициализация pymorphy
morph = MorphAnalyzer()

# Пример текста
texts = [
    "В случае развития сценария «Тысячелетний восторг» всему ценному персоналу необходимо назначить вторичных сотрудников.",
    "Эти вторичные сотрудники должны находиться в состоянии полной готовности.",
    "Они не исповедующих авраамических религий и проинформированы о местоположении главной копии плана действий «Патмос»."
]

# Список стоп-слов (можно расширить)
stop_words = set(["в", "и", "о", "с", "по", "на", "не", "они", "это", "для"])

Лемматизация, удаление стоп-слов

In [7]:
def preprocess_text(text):
    # Удаляем знаки препинания и приводим к нижнему регистру
    text = re.sub(r'[^\w\s]', '', text.lower())
    # Разбиваем текст на слова
    words = text.split()
    # Лемматизируем слова и удаляем стоп-слова
    lemmas = [morph.parse(word)[0].normal_form for word in words if word not in stop_words]
    return lemmas

# Применяем предобработку ко всем текстам
processed_texts = [preprocess_text(text) for text in texts]
print("Обработанные тексты:", processed_texts)

Обработанные тексты: [['случай', 'развитие', 'сценарий', 'тысячелетний', 'восторг', 'весь', 'ценный', 'персонал', 'необходимо', 'назначить', 'вторичный', 'сотрудник'], ['этот', 'вторичный', 'сотрудник', 'должный', 'находиться', 'состояние', 'полный', 'готовность'], ['исповедовать', 'авраамический', 'религия', 'проинформировать', 'местоположение', 'главный', 'копия', 'план', 'действие', 'патмос']]


Создание словаря

In [8]:
# Собираем все уникальные слова из всех текстов
vocabulary = set()
for text in processed_texts:
    vocabulary.update(text)

# Преобразуем словарь в список для удобства
vocabulary = list(vocabulary)
print("Словарь уникальных слов:", vocabulary)

Словарь уникальных слов: ['случай', 'патмос', 'сотрудник', 'должный', 'действие', 'тысячелетний', 'ценный', 'полный', 'весь', 'готовность', 'необходимо', 'этот', 'исповедовать', 'религия', 'развитие', 'восторг', 'назначить', 'персонал', 'проинформировать', 'план', 'местоположение', 'находиться', 'главный', 'авраамический', 'состояние', 'копия', 'сценарий', 'вторичный']


Функция для создания BoW

In [9]:
def build_bow(texts, vocabulary):
    # Создаем пустую матрицу BoW
    bow = []
    for text in texts:
        # Создаем вектор для текущего текста
        vector = [0] * len(vocabulary)
        for word in text:
            if word in vocabulary:
                # Увеличиваем счетчик для соответствующего слова
                index = vocabulary.index(word)
                vector[index] += 1
        bow.append(vector)
    return bow

# Создаем BoW
bow = build_bow(processed_texts, vocabulary)
print("Bag of Words (BoW):", bow)

Bag of Words (BoW): [[1, 0, 1, 0, 0, 1, 1, 0, 1, 0, 1, 0, 0, 0, 1, 1, 1, 1, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1], [0, 0, 1, 1, 0, 0, 0, 1, 0, 1, 0, 1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 1, 0, 0, 1], [0, 1, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0, 1, 1, 0, 0, 0, 0, 1, 1, 1, 0, 1, 1, 0, 1, 0, 0]]


Вычисление TF

In [10]:
def compute_tf(texts, vocabulary):
    tf = []
    for text in texts:
        vector = []
        for word in vocabulary:
            # Считаем частоту слова в тексте
            count = text.count(word)
            vector.append(count / len(text) if len(text) > 0 else 0)
        tf.append(vector)
    return tf

# Вычисляем TF
tf = compute_tf(processed_texts, vocabulary)
print("Term Frequency (TF):", tf)

Term Frequency (TF): [[0.08333333333333333, 0.0, 0.08333333333333333, 0.0, 0.0, 0.08333333333333333, 0.08333333333333333, 0.0, 0.08333333333333333, 0.0, 0.08333333333333333, 0.0, 0.0, 0.0, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.08333333333333333, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.08333333333333333, 0.08333333333333333], [0.0, 0.0, 0.125, 0.125, 0.0, 0.0, 0.0, 0.125, 0.0, 0.125, 0.0, 0.125, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.125, 0.0, 0.0, 0.125, 0.0, 0.0, 0.125], [0.0, 0.1, 0.0, 0.0, 0.1, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.1, 0.1, 0.0, 0.0, 0.0, 0.0, 0.1, 0.1, 0.1, 0.0, 0.1, 0.1, 0.0, 0.1, 0.0, 0.0]]


Вычисление IDF

In [12]:
def compute_idf(texts, vocabulary):
    idf = []
    total_docs = len(texts)
    for word in vocabulary:
        # Считаем количество документов, содержащих слово
        doc_count = sum(1 for text in texts if word in text)
        # Вычисляем IDF
        idf.append(math.log((total_docs + 1) / (doc_count + 1)) + 1)
    return idf

# Вычисляем IDF
idf = compute_idf(processed_texts, vocabulary)
print("Inverse Document Frequency (IDF):", idf)

Inverse Document Frequency (IDF): [1.6931471805599454, 1.6931471805599454, 1.2876820724517808, 1.6931471805599454, 1.6931471805599454, 1.6931471805599454, 1.6931471805599454, 1.6931471805599454, 1.6931471805599454, 1.6931471805599454, 1.6931471805599454, 1.6931471805599454, 1.6931471805599454, 1.6931471805599454, 1.6931471805599454, 1.6931471805599454, 1.6931471805599454, 1.6931471805599454, 1.6931471805599454, 1.6931471805599454, 1.6931471805599454, 1.6931471805599454, 1.6931471805599454, 1.6931471805599454, 1.6931471805599454, 1.6931471805599454, 1.6931471805599454, 1.2876820724517808]


Вычисление TF-IDF

In [13]:
def compute_tfidf(tf, idf):
    tfidf = []
    for tf_vector in tf:
        # Умножаем TF на IDF для каждого слова
        tfidf_vector = [tf_value * idf_value for tf_value, idf_value in zip(tf_vector, idf)]
        tfidf.append(tfidf_vector)
    return tfidf

# Вычисляем TF-IDF
tfidf = compute_tfidf(tf, idf)
print("TF-IDF:", tfidf)

TF-IDF: [[0.14109559837999544, 0.0, 0.10730683937098173, 0.0, 0.0, 0.14109559837999544, 0.14109559837999544, 0.0, 0.14109559837999544, 0.0, 0.14109559837999544, 0.0, 0.0, 0.0, 0.14109559837999544, 0.14109559837999544, 0.14109559837999544, 0.14109559837999544, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.14109559837999544, 0.10730683937098173], [0.0, 0.0, 0.1609602590564726, 0.21164339756999317, 0.0, 0.0, 0.0, 0.21164339756999317, 0.0, 0.21164339756999317, 0.0, 0.21164339756999317, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.21164339756999317, 0.0, 0.0, 0.21164339756999317, 0.0, 0.0, 0.1609602590564726], [0.0, 0.16931471805599455, 0.0, 0.0, 0.16931471805599455, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.16931471805599455, 0.16931471805599455, 0.0, 0.0, 0.0, 0.0, 0.16931471805599455, 0.16931471805599455, 0.16931471805599455, 0.0, 0.16931471805599455, 0.16931471805599455, 0.0, 0.16931471805599455, 0.0, 0.0]]
